## Preparación de los datos

A continuación, se preparan los datos que se utilizarán en los siguientes notebooks de esta práctica. El archivo ZIP que contiene las imágenes se descarga directamente desde la página de Kaggle.

Las imágenes incluidas en las carpetas `seg_train` y `seg_test` se descomprimen en la carpeta `data`, cada una en una subcarpeta correspondiente a su categoría.

Después, se crean dos arrays:
- El primero contiene las imágenes en formato RGB, redimensionadas a 150x150 píxeles (aunque en principio ya deberían tener ese tamaño) y normalizadas.
- El segundo array asocia a cada imagen una etiqueta numérica correspondiente a su categoría, con valores entre 0 y 5.

Finalmente, ambos arrays se guardan en archivos dentro de la carpeta principal de la práctica, para ser utilizados en los notebooks donde se crearán los modelos.


In [2]:
try:
    import cv2
except ImportError:
    import sys
    !{sys.executable} -m pip install opencv-python-headless
    import cv2

import os
import zipfile
import requests
from tqdm import tqdm
from pathlib import Path
import shutil
import warnings
import numpy as np

In [3]:
url = "https://www.kaggle.com/api/v1/datasets/download/puneet6060/intel-image-classification"
zip_path = "data.zip"
extract_dir = "data"

In [4]:
# Descargar el archivo zip con barra de progreso
def download_zip(url, output_path):
    response = requests.get(url, stream=True)
    total = int(response.headers.get('content-length', 0))
    with open(output_path, 'wb') as file, tqdm(
        desc=output_path,
        total=total,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in response.iter_content(chunk_size=1024):
            size = file.write(data)
            bar.update(size)

# Solo descargar si no existe
if not Path(zip_path).exists():
    print("Descargando archivo...")
    download_zip(url, zip_path)
else:
    print("Archivo ya descargado.")

# Crear carpeta de destino si no existe
os.makedirs(extract_dir, exist_ok=True)

Archivo ya descargado.


In [7]:
def extract_and_flatten(zip_path, target_folder):
    with zipfile.ZipFile(zip_path, 'r') as z:
        for member in z.infolist():
            # Procesamos solo seg_train y seg_test
            if ("seg_train/seg_train" in member.filename or "seg_test/seg_test" in member.filename) and member.filename.endswith('.jpg'):
                parts = Path(member.filename).parts
                if 'seg_train' in parts:
                    label = parts[2]
                    filename = parts[3]
                elif 'seg_test' in parts:
                    label = parts[2]
                    filename = parts[3]
                else:
                    continue

                # Convertimos a Path todo
                dest_dir = Path(target_folder) / label.lower()
                dest_dir.mkdir(parents=True, exist_ok=True)
                dest_path = dest_dir / filename

                # Comprobamos si existe antes de escribir
                if dest_path.exists():
                    warnings.warn(f"Overwriting file: {dest_path}")

                # Copiamos el archivo
                with z.open(member) as source, open(dest_path, "wb") as target:
                    shutil.copyfileobj(source, target)

# Ejecutar extracción
extract_and_flatten(zip_path, extract_dir)

print("Extracción completada. Directorio con las imagenes:", Path(extract_dir).resolve())

Extracción completada. Directorio con las imagenes: /opt/notebooks/M9/data


In [22]:
# Carpeta donde están las subcarpetas (buildings, forest, etc.)
data_dir = Path(extract_dir)
img_height, img_width = 150, 150

# Identificar las clases y asignarles un índice
classes = sorted([d.name for d in data_dir.iterdir() if d.is_dir()])
class_indices = {cls_name: idx for idx, cls_name in enumerate(classes)}
print("Clases encontradas:", class_indices)

# Listas para almacenar los datos
images = []
categories = []

# Procesar cada imagen
for cls_name in classes:
    cls_folder = data_dir / cls_name
    label = class_indices[cls_name]
    
    for img_path in tqdm(list(cls_folder.glob("*.jpg")), desc=f"Cargando {cls_name}"):
        img = cv2.imread(str(img_path))
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (img_width, img_height))
        img = img.astype('float32') / 255.0

        images.append(img)
        categories.append(label)

# Convertir a NumPy arrays
images = np.array(images, dtype=np.float32)
categories = np.array(categories, dtype=np.int32)

# Mostrar formas
print("Forma de images:", images.shape)
print("Forma de categories:", categories.shape)

# Guardar
np.save("images.npy", images)
np.save("categories.npy", categories)
print("Guardado en images.npy y categories.npy")

Clases encontradas: {'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}


Cargando buildings: 100% 2628/2628 [00:06<00:00, 423.75it/s]
Cargando forest: 100% 2745/2745 [00:06<00:00, 434.17it/s]
Cargando glacier: 100% 2957/2957 [00:06<00:00, 425.88it/s]
Cargando mountain: 100% 3037/3037 [00:06<00:00, 502.09it/s]
Cargando sea: 100% 2784/2784 [00:05<00:00, 479.86it/s]
Cargando street: 100% 2883/2883 [00:05<00:00, 532.41it/s]


Forma de images: (17034, 150, 150, 3)
Forma de categories: (17034,)
Guardado en images.npy y categories.npy
